In [ ]:
from model import Model
from dataset_loader import DatasetLoader
from dcdm import DCDM

import pandas as pd
import gc

import pickle

## 1. Loading datasets

Checklist:
- Each matrix should be of type `double`
- Every `nan` should be removed
- `y` should be flattened

In [ ]:
dl = DatasetLoader()

In [ ]:
dl.datasets

## 2. Experiments

In [ ]:
all_datasets = dl.datasets
optimizers = [
    DCDM, 
    'mosek', 
    'cvxpy_SCS'
]

In [ ]:
results = {}

for opt in optimizers:
    results[opt] = {}


for i, dataset_name in enumerate(dl.datasets):
    dataset_data = dl.get_dataset(dataset_name)
    for opt in optimizers:
        results[opt][dataset_name] = {}
        print(f'Fitting {dataset_name} on optimizer {opt}...')
        m = Model()
        result, df, time = m.fit(X=dataset_data['X_train'], y=dataset_data['y_train'], optimizer=opt)
        results[opt][dataset_name]['result'] = result
        results[opt][dataset_name]['df'] = df
        results[opt][dataset_name]['time'] = time
    del dataset_data
    gc.collect()
    
    with open(f'results_{i}.pickle', 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)